In [1]:
from datasets import load_dataset
import torch

# ds = load_dataset("cais/mmlu", "all", streaming=True)
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.2-3B')
model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-3.2-3B')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
stem_categories = [
    "abstract_algebra",
    "college_mathematics",
    "elementary_mathematics",
    "high_school_mathematics",
    "high_school_statistics",
    "astronomy",
    "college_physics",
    "conceptual_physics",
    "high_school_physics",
    "college_biology",
    "high_school_biology",
    "college_chemistry",
    "high_school_chemistry",
    "college_computer_science",
    "computer_security",
    "high_school_computer_science",
    "machine_learning",
    "electrical_engineering"
]
from datasets import concatenate_datasets, load_dataset
stem_data = []
for cat in stem_categories:
    stem_data.append(load_dataset('cais/mmlu', cat)['test'])
    
STEM = concatenate_datasets(stem_data)

In [3]:
humanities_categories = [ 
    "high_school_european_history",
    "high_school_us_history",
    "high_school_world_history",
    "prehistory",
    "moral_disputes",
    "moral_scenarios",
    "formal_logic",
    "logical_fallacies",
    "philosophy",
    "world_religions",
    "international_law",
    "jurisprudence",
    "professional_law"]
humanities_data = []
for cat in humanities_categories:
    humanities_data.append(load_dataset('cais/mmlu', cat)['test'])
    
HUMANITIES = concatenate_datasets(humanities_data)

In [4]:
social_science_categories = ["econometrics",
    "high_school_macroeconomics",
    "high_school_microeconomics",
    "high_school_geography",
    "human_sexuality",
    "sociology",
    "high_school_psychology",
    "professional_psychology",
    "high_school_government_and_politics",
    "public_relations",
    "security_studies",
    "us_foreign_policy",
    ]
social_science_data = []
for cat in social_science_categories:
    social_science_data.append(load_dataset('cais/mmlu', cat)['test'])
    
SOCIALSCIENCE = concatenate_datasets(social_science_data)

In [5]:
others_categories = [
    "anatomy",
    "clinical_knowledge",
    "college_medicine",
    "human_aging",
    "medical_genetics",
    "nutrition",
    "professional_medicine",
    "virology",
    "business_ethics",
    "management",
    "marketing",
    "professional_accounting",
    "global_facts",
    "miscellaneous"
]
others_data = []
for cat in others_categories:
    others_data.append(load_dataset('cais/mmlu', cat)['test'])
    
OTHERS = concatenate_datasets(others_data)

In [6]:
from peft import PeftModel, PeftConfig

peft_checkpoint_path = "checkpoint-99842"  # Path to your PEFT checkpoint

# Load PEFT configuration
peft_config = PeftConfig.from_pretrained(peft_checkpoint_path)

# Attach PEFT model to the base model
model = PeftModel.from_pretrained(model, peft_checkpoint_path)


In [7]:
from transformers import GenerationConfig
config = GenerationConfig(max_new_tokens=5)
model.to('cuda')

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Lin

In [11]:
base_prompt = 'Answer the following MCQ Question\n\n {question} based on Options given below\n\n Option1: {option1}\n Option2: {option2}\n Option3: {option3}\n Option4: {option4}\n\n  Answer:'

In [12]:
stem_matchh=0
i=0
from tqdm import tqdm
for item in tqdm(STEM):
    ans = tokenizer.decode(model.generate(**tokenizer(base_prompt.format(question=item['question'], option1 = item['choices'][0], option2 = item['choices'][1], option3 = item['choices'][2], option4 = item['choices'][3]), return_tensors='pt').to('cuda'), generation_config=config)[0, -3:-2])
    # print(f'Answer: {ans} True: {item["answer"]}')
    try:
        if int(ans) == item['answer']:
            stem_matchh+=1
        i+=1
    except:
        print('Answer is', ans)

  3%|▎         | 98/3018 [00:14<06:44,  7.22it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer is ,


  5%|▌         | 155/3018 [00:22<06:36,  7.23it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer is 




  9%|▉         | 274/3018 [00:38<06:52,  6.65it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

 73%|███████▎  | 2214/3018 [05:19<02:01,  6.63it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer is  based


 91%|█████████▏| 2758/3018 [06:45<00:41,  6.27it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer is  


100%|██████████| 3018/3018 [07:21<00:00,  6.83it/s]


In [ ]:
stem_matchh / len(STEM) 

0.46520874751491054

In [14]:
humanities_matchh=0
i=0
from tqdm import tqdm
for item in tqdm(HUMANITIES):
    ans = tokenizer.decode(model.generate(**tokenizer(base_prompt.format(question=item['question'], option1 = item['choices'][0], option2 = item['choices'][1], option3 = item['choices'][2], option4 = item['choices'][3]), return_tensors='pt').to('cuda'), generation_config=config)[0, -3:-2])
    # print(f'Answer: {ans} True: {item["answer"]}')
    try:
        if int(ans) == item['answer']:
            humanities_matchh+=1
        i+=1
    except:
        print(ans)

  0%|          | 0/4705 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
100%|██████████| 4705/4705 [14:02<00:00,  5.58it/s]


In [15]:
humanities_matchh / len(HUMANITIES)

0.4967056323060574

In [16]:
social_science_matchh=0
i=0
from tqdm import tqdm
for item in tqdm(SOCIALSCIENCE):
    ans = tokenizer.decode(model.generate(**tokenizer(base_prompt.format(question=item['question'], option1 = item['choices'][0], option2 = item['choices'][1], option3 = item['choices'][2], option4 = item['choices'][3]), return_tensors='pt').to('cuda'), generation_config=config)[0, -3:-2])
    # print(f'Answer: {ans} True: {item["answer"]}')
    try:
        if int(ans) == item['answer']:
            social_science_matchh+=1
        i+=1
    except:
        print(ans)

100%|██████████| 3077/3077 [07:26<00:00,  6.90it/s]


In [17]:
social_science_matchh / len(SOCIALSCIENCE)

0.6386090347741307

In [18]:
others_matchh=0
i=0
from tqdm import tqdm
for item in tqdm(OTHERS):
    ans = tokenizer.decode(model.generate(**tokenizer(base_prompt.format(question=item['question'], option1 = item['choices'][0], option2 = item['choices'][1], option3 = item['choices'][2], option4 = item['choices'][3]), return_tensors='pt').to('cuda'), generation_config=config)[0, -3:-2])
    # print(f'Answer: {ans} True: {item["answer"]}')
    try:
        if int(ans) == item['answer']:
            others_matchh+=1
        i+=1
    except:
        print(ans)

 11%|█         | 363/3242 [00:50<07:08,  6.72it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


 14%|█▍        | 447/3242 [01:03<07:19,  6.36it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


 90%|█████████ | 2932/3242 [06:58<00:42,  7.34it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


.


100%|██████████| 3242/3242 [07:39<00:00,  7.05it/s]


In [19]:
others_matchh / len(OTHERS)

0.6233806292412092

In [7]:
model = torch.load('nlp_model.pt')

/tmp/ipykernel_236038/708138291.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('nlp_model.pt')


In [11]:
base_prompt = 'Answer the following MCQ Question\n\n {question} based on Options given below\n\n Option1: {option1}\n Option2: {option2}\n Option3: {option3}\n Option4: {option4}\n\n  Answer:'

In [9]:
from transformers import GenerationConfig
config = GenerationConfig(max_new_tokens=5)

In [12]:
tokenizer.pad_token = tokenizer.eos_token

In [13]:
model.to('cuda')

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [14]:
stem_matchh=0
i=0
from tqdm import tqdm
for item in tqdm(STEM):
    ans = tokenizer.decode(model.generate(**tokenizer(base_prompt.format(question=item['question'], option1 = item['choices'][0], option2 = item['choices'][1], option3 = item['choices'][2], option4 = item['choices'][3]), return_tensors='pt').to('cuda'), generation_config=config)[0, -3:-2])
    # print(f'Answer: {ans} True: {item["answer"]}')
    try:
        if int(ans) == item['answer']:
            stem_matchh+=1
        i+=1
    except:
        print('Answer is', ans)

  0%|          | 1/3018 [00:00<21:46,  2.31it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 3/3018 [00:00<08:17,  6.06it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 5/3018 [00:00<05:51,  8.56it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 7/3018 [00:00<04:52, 10.28it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 9/3018 [00:00<04:20, 11.57it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 11/3018 [00:01<04:02, 12.38it/s]S

In [15]:
stem_matchh / len(STEM)

0.33465871438038436

In [16]:
humanities_matchh=0
i=0
from tqdm import tqdm
for item in tqdm(HUMANITIES):
    ans = tokenizer.decode(model.generate(**tokenizer(base_prompt.format(question=item['question'], option1 = item['choices'][0], option2 = item['choices'][1], option3 = item['choices'][2], option4 = item['choices'][3]), return_tensors='pt').to('cuda'), generation_config=config)[0, -3:-2])
    # print(f'Answer: {ans} True: {item["answer"]}')
    try:
        if int(ans) == item['answer']:
            humanities_matchh+=1
        i+=1
    except:
        print(ans)

  0%|          | 1/4705 [00:00<10:41,  7.33it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 4/4705 [00:00<08:36,  9.11it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 6/4705 [00:00<08:17,  9.45it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 9/4705 [00:00<08:20,  9.38it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 11/4705 [00:01<07:48, 10.03it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 14/4705 [00:01<08:12,  9.52it/s]

In [17]:
humanities_matchh / len(HUMANITIES)

0.37194473963868224

In [18]:
social_science_matchh=0
i=0
from tqdm import tqdm
for item in tqdm(SOCIALSCIENCE):
    ans = tokenizer.decode(model.generate(**tokenizer(base_prompt.format(question=item['question'], option1 = item['choices'][0], option2 = item['choices'][1], option3 = item['choices'][2], option4 = item['choices'][3]), return_tensors='pt').to('cuda'), generation_config=config)[0, -3:-2])
    # print(f'Answer: {ans} True: {item["answer"]}')
    try:
        if int(ans) == item['answer']:
            social_science_matchh+=1
        i+=1
    except:
        print(ans)

  0%|          | 0/3077 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 2/3077 [00:00<03:11, 16.03it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 4/3077 [00:00<03:06, 16.48it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 6/3077 [00:00<03:06, 16.44it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 8/3077 [00:00<03:03, 16.72it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 10/3077 [00:00<03:02, 16.80it/s]Setting `

In [19]:
social_science_matchh / len(SOCIALSCIENCE)

0.41501462463438415

In [20]:
others_matchh=0
i=0
from tqdm import tqdm
for item in tqdm(OTHERS):
    ans = tokenizer.decode(model.generate(**tokenizer(base_prompt.format(question=item['question'], option1 = item['choices'][0], option2 = item['choices'][1], option3 = item['choices'][2], option4 = item['choices'][3]), return_tensors='pt').to('cuda'), generation_config=config)[0, -3:-2])
    # print(f'Answer: {ans} True: {item["answer"]}')
    try:
        if int(ans) == item['answer']:
            others_matchh+=1
        i+=1
    except:
        print(ans)

  0%|          | 0/3242 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 2/3242 [00:00<03:09, 17.09it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 4/3242 [00:00<02:56, 18.35it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 6/3242 [00:00<03:00, 17.94it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 8/3242 [00:00<02:57, 18.17it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 10/3242 [00:00<02:59, 17.99it/s]Setting `

In [21]:
others_matchh / len(OTHERS)

0.3985194324491055

In [11]:
matchh/i

0.5610711952971914

1530